# Value Investing with Python - Part 04

Part four of the **Value Investing with Python**-series deals with **fundamental analysis** as it provides a function to evaluate the previously obtained metrics included in the Morningstar data. The evaluation is compressed into a standardized scheme.

The function has one input: the modified Morningstar data (see the **updated** part two).

The function outputs two DataFrame objects containing the evaluation results.

In [1]:
def metric_assessment(stock_data_df):
    
    # Function to assess the development of relevant metrics by period (1 year, 3 years, 10 years)
    
    # Input: modified Pandas DataFrame containing financial and key ratios data webscraped from Morningstar.com
    # Output: 
    ## growth_rate_df: Pandas DataFrame for comparisons of growth rates of metrics
    ## median_value_df: Pandas DataFrame for comparisons of median values of metrics
    
    # Some metrics (e.g. revenue or net income) are assessed by looking at the growth rates,
    # while others (e.g. return on equity) are assessed by their median values.
    
    # Required modules
    import pandas as pd
    import numpy as np
    
    # Define metrics to assess:    
    growth_rate_metrics = ['revenue_mil', 'operating_income_mil', 'net_income_mil', 'eps', 'dividends', 'bvps', 
                           'operating_cashflow_mil', 'free_cashflow_mil', 'cap_ex_mil'] 
    
    median_value_metrics = ['payout_ratio_pct', 'interest_coverage_ratio', 'operating_margin_pct', 'net_margin_pct',
                            'gross_margin_pct', 'return_on_equity_pct', 'return_on_assets_pct',
                            'return_on_invested_capital_pct', 'free_cashflow_to_revenue_pct', 'current_ratio',
                            'debt_to_equity_ratio']

    # 1. Growth rate metrics:
    # Define periods (last period = 9, because only 9 growth rates for a 10 year period are available):
    periods = [9, 3, 1]
        
    # Create dictionary to collect metric name and corresponding values:
    growth_rate_dict = {key: [] for key in growth_rate_metrics}
    
    for metric in growth_rate_metrics:
        # Create series containing the metric's values:
        if metric == "cap_ex_mil":
            metric_series = stock_data_df[metric] * (-1)
        else:
            metric_series = stock_data_df[metric]
        
        for period in periods:
            # Create list containing years (as numbers, e.g. 2012 = year 1 = 1, 2013 = year 2 = 2 etc.) included in the period:
            years_numbers = [len(stock_data_df[metric].index) - year for year in reversed(range(period + 1))]
            
            # Create empty list to collect growth rates:
            growth_rates = []
            
            # Loop through all combinations of years:
            step = 0

            while period > 0:
                for year in years_numbers[:-1]:

                    # Determine indices (start_year - 1 to adjust for the fact that indices start at 0):
                    start_index = year - 1
                    end_index = year + step

                    # Determine values at the specific indices:
                    start_value = metric_series[start_index]
                    end_value = metric_series[end_index]
                    
                    # Check for zero values:
                    if np.min([start_value, end_value]) <= 0:
                        growth_rates.append(np.nan)
                        continue

                    # Calculate difference between years:
                    diff = float(metric_series.index[end_index]) - float(metric_series.index[start_index])

                    # Calculate compound annual growth rate (CAGR):
                    cagr = (end_value / start_value) ** (1 / diff)

                    growth_rates.append(cagr-1)

                years_numbers.pop(-1)
                step += 1
                period -= 1
            
            # Compute median CAGR for period. If more than 50% of datapoints are NaN, growth rate = NaN:
            if pd.isna(growth_rates).sum() < len(growth_rates) * 0.5:
                median_gr = round(float(np.nanmedian(growth_rates) * 100), 2)
            else:
                median_gr = np.nan
        
            # Append median CAGR to dictionary:
            growth_rate_dict[metric].append(median_gr)
            
    # 2. Median value metrics:
    # Define periods:
    periods = [10, 3, 1]
    
    # Create dictionary to collect metric name and corresponding values:
    median_value_dict = {key: [] for key in median_value_metrics}
    
    for metric in median_value_metrics:
        # Create series containing the metric's values:
        metric_series = stock_data_df[metric]
        
        for period in periods:
            # If more than 50% of datapoints are NaN, median value = NaN:
            if pd.isna(metric_series[-(period):]).sum() < len(metric_series[-(period):]) * 0.5:
                median_value = round(float(np.nanmedian(metric_series[-(period):])), 2)
            else:
                median_value = np.nan

            # Append median dictionary:
            median_value_dict[metric].append(median_value)
            
    # Convert both dictionaries to pandas Dataframes:
    growth_rate_df = pd.DataFrame(growth_rate_dict.values(), index=growth_rate_dict.keys(), columns=['10Y', '3Y', '1Y'])
    median_value_df = pd.DataFrame(median_value_dict.values(), index=median_value_dict.keys(), columns=['10Y', '3Y', '1Y'])
    
    # 3. Assign scores/ points to the figures:
    # 3.1. Growth rate metrics: 1 point, if positive (>0). Maximum points: 3.
    growth_rate_df['points'] = growth_rate_df.apply(lambda row: np.sum(row > 0), axis = 1)
    
    # 3.2. Median value metrics
    points_median_value = []
    
    # Payout ratio: 1, if value < 80%. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['payout_ratio_pct', :] < 80))
    
    # Interest Coverage Ratio: 1, if value > 1.5. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['interest_coverage_ratio', :] > 1.5))  
    
    # Operating margin, net margin, gross margin: 1, if value > 10%. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['operating_margin_pct', :] > 10))
    points_median_value.append(np.sum(median_value_df.loc['net_margin_pct', :] > 10))
    points_median_value.append(np.sum(median_value_df.loc['gross_margin_pct', :] > 10))
        
    # Return on Equity: 1, if value > 8%. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['return_on_equity_pct', :] > 8)) 

    # Return on Assets: 1, if value > 6%. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['return_on_assets_pct', :] > 8)) 
    
    # Return on Invested Capital: 1, if value > 8%. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['return_on_invested_capital_pct', :] > 8))
    
    # Free Cashflow to Revenue: 1, if value > 5%. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['free_cashflow_to_revenue_pct', :] > 5))
    
    # Current Ratio: 1, if value > 1. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['current_ratio', :] > 1))    

    # Debt/Equity Ratio: 1, if value < 1. Maximum points: 3.
    points_median_value.append(np.sum(median_value_df.loc['debt_to_equity_ratio', :] < 1))
    
    # Assign the points to the dataframe:
    median_value_df["points"] = points_median_value
    
    return growth_rate_df, median_value_df

To illustrate an example, one has to use the function from part two:

In [2]:
import nbimporter
import datetime

In [3]:
from VIPy_02_GetModifyMorningstarData import *

In [4]:
apple_data = get_modify_morningstar_data("AAPL", "XNAS")

In [5]:
apple_growthrates_df, apple_median_df = metric_assessment(apple_data)

In [6]:
apple_growthrates_df

,10Y,3Y,1Y,points
revenue_mil,7.84,8.39,33.26,3
operating_income_mil,4.41,9.54,64.36,3
net_income_mil,6.89,10.31,64.92,3
eps,12.62,16.96,71.04,3
dividends,10.34,6.84,6.17,3
bvps,0.92,-14.32,-8.22,1
operating_cashflow_mil,6.00,13.30,28.96,3
free_cashflow_mil,7.40,18.87,26.70,3
cap_ex_mil,1.85,-13.54,51.66,2


In [7]:
apple_median_df

,10Y,3Y,1Y,points
payout_ratio_pct,24.73,24.24,15.15,3
interest_coverage_ratio,42.29,24.35,42.29,3
operating_margin_pct,28.26,24.57,29.78,3
net_margin_pct,21.64,21.24,25.88,3
gross_margin_pct,38.53,38.23,41.78,3
return_on_equity_pct,44.55,73.69,147.44,3
return_on_assets_pct,17.67,17.33,28.06,3
return_on_invested_capital_pct,26.14,30.11,51.70,3
free_cashflow_to_revenue_pct,25.75,25.41,25.41,3
current_ratio,1.31,1.36,1.07,3
